In [659]:
import requests
from requests import get
from bs4 import BeautifulSoup
import pandas as pd

import os
from pprint import pprint

#using this to prevent a site from kicking us out because it acts more like a human and less like an automated process
import time

import re
import json

## * Get Access *

In [666]:
#attempting to access the website - blocked because it sees a machine and not a human
url = "https://codeup.com/blog/"
response = get(url)
response

<Response [403]>

In [667]:
#check the header because some sites do not allow python-requests
requests.utils.default_headers()

{'User-Agent': 'python-requests/2.28.1', 'Accept-Encoding': 'gzip, deflate, br', 'Accept': '*/*', 'Connection': 'keep-alive'}

In [668]:
#assign new user_agent
user_agent = ["Mozilla/5.0 (Windows NT 10.0; rv:91.0) Gecko/20100101 Firefox/91.0"]

In [669]:
#changing the header from python to Chrome
headers = {"User-Agent": "Chrome/91.0.4472.124"}
response = get(url, headers=headers)
response

<Response [200]>

## 1. 5 distinct blog posts. For each post, you should scrape at least the post's title and content.
* Encapsulate your work in a function named get_blog_articles that will return a list of dictionaries, with each dictionary representing one article.

In [670]:
#make a soup variable holding the response content
soup = BeautifulSoup(response.content, 'html.parser')
soup

<!DOCTYPE html>

<html lang="en-US">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<link href="https://codeup.com/xmlrpc.php" rel="pingback"/>
<script type="text/javascript">
		document.documentElement.className = 'js';
	</script>
<link crossorigin="" href="https://fonts.gstatic.com" rel="preconnect"/><script id="diviarea-loader">window.DiviPopupData=window.DiviAreaConfig={"zIndex":1000000,"animateSpeed":400,"triggerClassPrefix":"show-popup-","idAttrib":"data-popup","modalIndicatorClass":"is-modal","blockingIndicatorClass":"is-blocking","defaultShowCloseButton":true,"withCloseClass":"with-close","noCloseClass":"no-close","triggerCloseClass":"close","singletonClass":"single","darkModeClass":"dark","noShadowClass":"no-shadow","altCloseClass":"close-alt","popupSelector":".et_pb_section.popup","initializeOnEvent":"et_pb_after_init_modules","popupWrapperClass":"area-outer-wrap","fullHeightClass":"full-height","openPopupClass":"da-overlay-visible","ove

In [671]:
#gave the html script for the webpage title element
soup.h1

<h1>Codeup Blog</h1>

In [287]:
#pulls back all the headers and searched for the class_ name pulled from the inspection of the website for entry-title
article_links = soup.find_all('h2', class_='entry-title')
article_links

[<h2 class="entry-title"><a href="https://codeup.com/featured/apida-heritage-month/">Spotlight on APIDA Voices: Celebrating Heritage and Inspiring Change ft. Arbeena Thapa</a></h2>,
 <h2 class="entry-title"><a href="https://codeup.com/featured/women-in-tech-panelist-spotlight/">Women in tech: Panelist Spotlight – Magdalena Rahn</a></h2>,
 <h2 class="entry-title"><a href="https://codeup.com/featured/women-in-tech-rachel-robbins-mayhill/">Women in tech: Panelist Spotlight – Rachel Robbins-Mayhill</a></h2>,
 <h2 class="entry-title"><a href="https://codeup.com/codeup-news/women-in-tech-panelist-spotlight-sarah-mellor/">Women in Tech: Panelist Spotlight – Sarah Mellor</a></h2>,
 <h2 class="entry-title"><a href="https://codeup.com/events/women-in-tech-madeleine/">Women in Tech: Panelist Spotlight – Madeleine Capper</a></h2>,
 <h2 class="entry-title"><a href="https://codeup.com/codeup-news/panelist-spotlight-4/">Black Excellence in Tech: Panelist Spotlight – Wilmarie De La Cruz Mejia</a></h2>

In [288]:
#parse 1st link only and pull back title
article_links[0].text

'Spotlight on APIDA Voices: Celebrating Heritage and Inspiring Change ft. Arbeena Thapa'

In [289]:
#for loop to iterate and print the title of each element
title_list=[]

#for loop to iterate and print content of each element
num_elementst = len(article_links)


#using np.arange to iterate through soup skipping every other line (it was a date, didn't need it)
for i in np.arange(num_elementst):
    title_list.append(article_links[i].text)
title_list

['Spotlight on APIDA Voices: Celebrating Heritage and Inspiring Change ft. Arbeena Thapa',
 'Women in tech: Panelist Spotlight – Magdalena Rahn',
 'Women in tech: Panelist Spotlight – Rachel Robbins-Mayhill',
 'Women in Tech: Panelist Spotlight – Sarah Mellor',
 'Women in Tech: Panelist Spotlight – Madeleine Capper',
 'Black Excellence in Tech: Panelist Spotlight – Wilmarie De La Cruz Mejia']

In [290]:
#content locations are 1, 3, 5, 7, 9, 11
soup.find_all('p')[1].text

'May is traditionally known as Asian American and Pacific Islander (AAPI) Heritage Month. This month we celebrate the...'

In [291]:
#empty list
content_list=[]

#for loop to iterate and print content of each element
num_elements = len(soup.find_all('p'))

#using np.arange to iterate through soup skipping every other line (it was a date, didn't need it)
for i in np.arange(num_elements - 12, step=2):
    content_list.append(soup.find_all('p')[i+1].text)
content_list

['May is traditionally known as Asian American and Pacific Islander (AAPI) Heritage Month. This month we celebrate the...',
 'Women in tech: Panelist Spotlight – Magdalena Rahn Codeup is hosting a Women in Tech Panel in honor of Women’s History...',
 'Women in tech: Panelist Spotlight - Rachel Robbins-Mayhill Codeup is hosting a Women in Tech Panel in honor of Women’s...',
 'Women in tech: Panelist Spotlight - Sarah Mellor\xa0 Codeup is hosting a Women in Tech Panel in honor of Women’s History...',
 'Women in tech: Panelist Spotlight - Madeleine Capper Codeup is hosting a Women in Tech Panel in honor of Women’s...',
 'Black excellence in tech: Panelist Spotlight - Wilmarie De La Cruz Mejia Codeup is hosting a Black Excellence in Tech...']

In [292]:
#grabbing the date becuase why not
date_list=[]

#for loop to iterate and print content of each element
num_elementsd = len(soup.find_all('p'))

for i in np.arange(num_elementsd - 12, step=2):
    date_list.append(soup.find_all('p')[i].text)
date_list

['May 24, 2023',
 'Mar 28, 2023',
 'Mar 20, 2023',
 'Mar 13, 2023',
 'Mar 6, 2023',
 'Feb 16, 2023']

In [293]:
# format for the dictionary
# {
#     'title': 'the title of the article',
#     'content': 'the full text content of the article',
#     'date' : 'added for extra practice'
# }


In [294]:
#combine lists to one dictionary following assigned format
results = {'title':title_list
          , 'content':content_list
          , 'date' : date_list}
results

{'title': ['Spotlight on APIDA Voices: Celebrating Heritage and Inspiring Change ft. Arbeena Thapa',
  'Women in tech: Panelist Spotlight – Magdalena Rahn',
  'Women in tech: Panelist Spotlight – Rachel Robbins-Mayhill',
  'Women in Tech: Panelist Spotlight – Sarah Mellor',
  'Women in Tech: Panelist Spotlight – Madeleine Capper',
  'Black Excellence in Tech: Panelist Spotlight – Wilmarie De La Cruz Mejia'],
 'content': ['May is traditionally known as Asian American and Pacific Islander (AAPI) Heritage Month. This month we celebrate the...',
  'Women in tech: Panelist Spotlight – Magdalena Rahn Codeup is hosting a Women in Tech Panel in honor of Women’s History...',
  'Women in tech: Panelist Spotlight - Rachel Robbins-Mayhill Codeup is hosting a Women in Tech Panel in honor of Women’s...',
  'Women in tech: Panelist Spotlight - Sarah Mellor\xa0 Codeup is hosting a Women in Tech Panel in honor of Women’s History...',
  'Women in tech: Panelist Spotlight - Madeleine Capper Codeup is hos

In [295]:
#turn to df
results_df = pd.DataFrame(results)
results_df

,title,content,date
0,Spotlight on APIDA Voices: Celebrating Heritag...,May is traditionally known as Asian American a...,"May 24, 2023"
1,Women in tech: Panelist Spotlight – Magdalena ...,Women in tech: Panelist Spotlight – Magdalena ...,"Mar 28, 2023"
2,Women in tech: Panelist Spotlight – Rachel Rob...,Women in tech: Panelist Spotlight - Rachel Rob...,"Mar 20, 2023"
3,Women in Tech: Panelist Spotlight – Sarah Mellor,Women in tech: Panelist Spotlight - Sarah Mell...,"Mar 13, 2023"
4,Women in Tech: Panelist Spotlight – Madeleine ...,Women in tech: Panelist Spotlight - Madeleine ...,"Mar 6, 2023"
5,Black Excellence in Tech: Panelist Spotlight –...,Black excellence in tech: Panelist Spotlight -...,"Feb 16, 2023"


In [296]:
def get_blog_articles(url):
    '''
    this function grabs title, content, and date from Codeup Blog site using new user_agent that is not a python request
    '''
    #assign new user_agent
    user_agent = ["Mozilla/5.0 (Windows NT 10.0; rv:91.0) Gecko/20100101 Firefox/91.0"]
    
    #changing the header from python to Chrome
    headers = {"User-Agent": "Chrome/91.0.4472.124"}
    response = get(url, headers=headers)
    
    #make a soup variable holding the response content
    soup = BeautifulSoup(response.content, 'html.parser')
    
    #pulls back all the headers and searched for the class_ name pulled from the inspection of the website for entry-title
    article_links = soup.find_all('h2', class_='entry-title')

    #empty lists
    title_list=[]
    content_list=[]
    date_list = []
    
    #for loop to iterate and print content of each element
    num_elementst = len(article_links)
    num_elements = len(soup.find_all('p'))
    num_elementsd = len(soup.find_all('p'))

    #using np.arange to iterate through soup skipping every other line
    for i in np.arange(num_elementst):
        title_list.append(article_links[i].text)
    for i in np.arange(num_elements - 12, step=2):
        content_list.append(soup.find_all('p')[i+1].text)
    for i in np.arange(num_elementsd - 12, step=2):
        date_list.append(soup.find_all('p')[i].text)
        
    #combine lists to one dictionary following assigned format
    results = {'title':title_list
              ,'content':content_list
              ,'date' : date_list}
    
    #turn to df
    results_df = pd.DataFrame(results)
    
    return results_df


In [297]:
results_df = get_blog_articles(url)
results_df

,title,content,date
0,Spotlight on APIDA Voices: Celebrating Heritag...,May is traditionally known as Asian American a...,"May 24, 2023"
1,Women in tech: Panelist Spotlight – Magdalena ...,Women in tech: Panelist Spotlight – Magdalena ...,"Mar 28, 2023"
2,Women in tech: Panelist Spotlight – Rachel Rob...,Women in tech: Panelist Spotlight - Rachel Rob...,"Mar 20, 2023"
3,Women in Tech: Panelist Spotlight – Sarah Mellor,Women in tech: Panelist Spotlight - Sarah Mell...,"Mar 13, 2023"
4,Women in Tech: Panelist Spotlight – Madeleine ...,Women in tech: Panelist Spotlight - Madeleine ...,"Mar 6, 2023"
5,Black Excellence in Tech: Panelist Spotlight –...,Black excellence in tech: Panelist Spotlight -...,"Feb 16, 2023"


### Amanda's Way

In [673]:
#after inspecting the website - scrub for all anchors and the more-link to get the content
more_links = soup.find_all('a', class_='more-link')
more_links

[<a class="more-link" href="https://codeup.com/featured/apida-heritage-month/">read more</a>,
 <a class="more-link" href="https://codeup.com/featured/women-in-tech-panelist-spotlight/">read more</a>,
 <a class="more-link" href="https://codeup.com/featured/women-in-tech-rachel-robbins-mayhill/">read more</a>,
 <a class="more-link" href="https://codeup.com/codeup-news/women-in-tech-panelist-spotlight-sarah-mellor/">read more</a>,
 <a class="more-link" href="https://codeup.com/events/women-in-tech-madeleine/">read more</a>,
 <a class="more-link" href="https://codeup.com/codeup-news/panelist-spotlight-4/">read more</a>]

In [675]:
#extract the links into a list for each article
links_list = [link['href'] for link in more_links]
links_list

['https://codeup.com/featured/apida-heritage-month/',
 'https://codeup.com/featured/women-in-tech-panelist-spotlight/',
 'https://codeup.com/featured/women-in-tech-rachel-robbins-mayhill/',
 'https://codeup.com/codeup-news/women-in-tech-panelist-spotlight-sarah-mellor/',
 'https://codeup.com/events/women-in-tech-madeleine/',
 'https://codeup.com/codeup-news/panelist-spotlight-4/']

In [588]:
#getting the actual document html
soup = BeautifulSoup(response.content, 'html.parser')
soup

<!DOCTYPE html>

<html lang="en-US">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<link href="https://codeup.com/xmlrpc.php" rel="pingback"/>
<script type="text/javascript">
		document.documentElement.className = 'js';
	</script>
<link crossorigin="" href="https://fonts.gstatic.com" rel="preconnect"/><script id="diviarea-loader">window.DiviPopupData=window.DiviAreaConfig={"zIndex":1000000,"animateSpeed":400,"triggerClassPrefix":"show-popup-","idAttrib":"data-popup","modalIndicatorClass":"is-modal","blockingIndicatorClass":"is-blocking","defaultShowCloseButton":true,"withCloseClass":"with-close","noCloseClass":"no-close","triggerCloseClass":"close","singletonClass":"single","darkModeClass":"dark","noShadowClass":"no-shadow","altCloseClass":"close-alt","popupSelector":".et_pb_section.popup","initializeOnEvent":"et_pb_after_init_modules","popupWrapperClass":"area-outer-wrap","fullHeightClass":"full-height","openPopupClass":"da-overlay-visible","ove

In [693]:
#loop through the link to collect all of the articles' info
headers = {'User-Agent': 'Codeup Data Science'}
article_info = []

for link in links_list:
    response = get(link, headers=headers)
    
    soup = BeautifulSoup(response.content, 'html.parser')
    
    info_dict = {"title":soup.find("h1").text,
                "link": link,
                "date_published":soup.find('span', class_="published").text,
                "content": soup.find('div', class_="entry-content").text.replace('\n',' ')}
    article_info.append(info_dict)
    
article_info[0]


{'title': 'Spotlight on APIDA Voices: Celebrating Heritage and Inspiring Change ft. Arbeena Thapa',
 'link': 'https://codeup.com/featured/apida-heritage-month/',
 'date_published': 'May 24, 2023',
 'content': ' May is traditionally known as Asian American and Pacific Islander (AAPI) Heritage Month. This month we celebrate the history and contributions made possible by our AAPI friends, family, and community. We also examine our level of support and seek opportunities to better understand the AAPI community.  In an effort to address real concerns and experiences, we sat down with Arbeena Thapa, one of Codeup’s Financial Aid and Enrollment Managers. Arbeena identifies as Nepali American and Desi. Arbeena’s parents immigrated to Texas in 1988 for better employment and educational opportunities. Arbeena’s older sister was five when they made the move to the US. Arbeena was born later, becoming the first in her family to be a US citizen. At Codeup we take our efforts at inclusivity very ser

In [ ]:
#getting the title from new soup
title = soup.find('h1')
title.text

'Women in Tech: Panelist Spotlight – Sarah Mellor'

In [593]:
#getting the date
pdate = soup.find('span', class_='published')
pdate.text

'Mar 13, 2023'

In [703]:
#getting the entry content
artcont = soup.find('div', class_='entry-content')
artcont.text.replace('\n', ' ')

' Black excellence in tech: Panelist Spotlight – Wilmarie De La Cruz Mejia  Codeup is hosting a Black Excellence in Tech Panel in honor of Black History Month on February 22, 2023! To further celebrate, we’d like to spotlight each of our panelists leading up to the discussion to learn a bit about their respective experiences as black leaders in the tech industry!\xa0\xa0 Meet Wilmarie! Wilmarie De\xa0La Cruz Mejia is a current Codeup student on the path to becoming a Full-Stack Web Developer at our Dallas, TX campus.\xa0 Wilmarie is a veteran expanding her knowledge of programming languages and technologies on her journey with Codeup.\xa0 We asked Wilmarie to share more about her experience at Codeup. She shares, “I was able to meet other people who were passionate about coding and be in a positive learning environment.” We hope you can join us on February 22nd to sit in on an insightful conversation with Wilmarie and all of our panelists! '

In [704]:
#loop meow
article_info = []
#assign new user_agent
user_agent = ["Mozilla/5.0 (Windows NT 10.0; rv:91.0) Gecko/20100101 Firefox/91.0"]

#changing the header from python to Chrome
headers = {"User-Agent": "Chrome/91.0.4472.124"}

for link in links_list:
    # response
    response = get(link, headers=headers)
    # to get the soup
    soup = BeautifulSoup(response.content, 'html.parser')
    #dict
    info_dict = {'title':soup.find('h1').text,
                 'link':link,
                 'content':soup.find('div', class_='entry-content').text.replace('\n', ' ' ),
                 'date' : soup.find('span', class_='published').text}
    
    article_info.append(info_dict)
    
article_info

[{'title': 'Spotlight on APIDA Voices: Celebrating Heritage and Inspiring Change ft. Arbeena Thapa',
  'link': 'https://codeup.com/featured/apida-heritage-month/',
  'content': ' May is traditionally known as Asian American and Pacific Islander (AAPI) Heritage Month. This month we celebrate the history and contributions made possible by our AAPI friends, family, and community. We also examine our level of support and seek opportunities to better understand the AAPI community.  In an effort to address real concerns and experiences, we sat down with Arbeena Thapa, one of Codeup’s Financial Aid and Enrollment Managers. Arbeena identifies as Nepali American and Desi. Arbeena’s parents immigrated to Texas in 1988 for better employment and educational opportunities. Arbeena’s older sister was five when they made the move to the US. Arbeena was born later, becoming the first in her family to be a US citizen. At Codeup we take our efforts at inclusivity very seriously. After speaking with Arbe

In [696]:
def get_blog_articles_2(article_list):
    '''
    this function checks for a json file of scraped data. If no file exists, it scrapes and runs
    the function again then saves to a json in local directoty. returns df of items.
    '''
    file = "blog_posts.json"
    
    if os.path.exists(file):
        with open(file) as f:
            return json.load(f)
        
    
    headers = {'User-Agent': 'Codeup Data Science'}
    article_info = []

    for article in article_list:
        response = get(article, headers=headers)

        soup = BeautifulSoup(response.content, 'html.parser')

        info_dict = {"title":soup.find("h1").text,
                    "link": article,
                    "date_published":soup.find('span', class_="published").text,
                    "content": soup.find('div', class_="entry-content").text.replace('\n',' ')}
        article_info.append(info_dict)
    
    with open(file, "w") as f:
        json.dump(article_info, f)
        
    return article_info


In [700]:
article_info = pd.DataFrame(get_blog_articles_2(links_list))
article_info

,title,link,date_published,content
0,Spotlight on APIDA Voices: Celebrating Heritag...,https://codeup.com/featured/apida-heritage-month/,"May 24, 2023",May is traditionally known as Asian American ...
1,Women in tech: Panelist Spotlight – Magdalena ...,https://codeup.com/featured/women-in-tech-pane...,"Mar 28, 2023",Women in tech: Panelist Spotlight – Magdalena...
2,Women in tech: Panelist Spotlight – Rachel Rob...,https://codeup.com/featured/women-in-tech-rach...,"Mar 20, 2023",Women in tech: Panelist Spotlight – Rachel Ro...
3,Women in Tech: Panelist Spotlight – Sarah Mellor,https://codeup.com/codeup-news/women-in-tech-p...,"Mar 13, 2023",Women in tech: Panelist Spotlight – Sarah Mel...
4,Women in Tech: Panelist Spotlight – Madeleine ...,https://codeup.com/events/women-in-tech-madele...,"Mar 6, 2023",Women in tech: Panelist Spotlight – Madeleine...
5,Black Excellence in Tech: Panelist Spotlight –...,https://codeup.com/codeup-news/panelist-spotli...,"Feb 16, 2023",Black excellence in tech: Panelist Spotlight ...


## * Get Access *

In [705]:
#attempting to access the website - not blocked, accepts python requests
url = 'https://inshorts.com/en/read'
response = get(url)
response

<Response [200]>

## 2. We will now be scraping text data from inshorts, a website that provides a brief overview of many different topics.
* Write a function called get_news_articles that scrapes the news articles for the following topics:
    * Business
    * Sports
    * Technology
    * Entertainment

In [706]:
# url = 'https://inshorts.com/en/business'
# response = get(url)
# response

In [707]:
#make a soup variable holding the response content
soup = BeautifulSoup(response.content, 'html.parser')
soup

<!DOCTYPE html>

<html lang="en">
<head>
<meta charset="utf-8"/>
<style>
    /* The Modal (background) */
    .modal_contact {
        display: none; /* Hidden by default */
        position: fixed; /* Stay in place */
        z-index: 8; /* Sit on top */
        left: 0;
        top: 0;
        width: 100%; /* Full width */
        height: 100%;
        overflow: auto; /* Enable scroll if needed */
        background-color: rgb(0,0,0); /* Fallback color */
        background-color: rgba(0,0,0,0.4); /* Black w/ opacity */
    }

    /* Modal Content/Box */
    .modal-content {
        background-color: #fefefe;
        margin: 15% auto;
        padding: 20px !important;
        padding-top: 0 !important;
        /* border: 1px solid #888; */
        text-align: center;
        position: relative;
        border-radius: 6px;
    }

    /* The Close Button */
    .close {
      left: 90%;
      color: #aaa;
      float: right;
      font-size: 28px;
      font-weight: bold;
    /* positi

In [708]:
#after inspecting the website - scrub for all li to find Business, Sports, Technology, Entertainment...
soup.find_all('li')

[<li class="active-category selected">All News</li>,
 <li class="active-category">India</li>,
 <li class="active-category">Business</li>,
 <li class="active-category">Sports</li>,
 <li class="active-category">World</li>,
 <li class="active-category">Politics</li>,
 <li class="active-category">Technology</li>,
 <li class="active-category">Startup</li>,
 <li class="active-category">Entertainment</li>,
 <li class="active-category">Miscellaneous</li>,
 <li class="active-category">Hatke</li>,
 <li class="active-category">Science</li>,
 <li class="active-category">Automobile</li>]

In [709]:
category_links = soup.find_all('a')
category_links

[<a href="/"><img class="logo" src="https://assets.inshorts.com/website_assets/images/logo_inshorts.png"/></a>,
 <a href="/en/read"><div class="lang-en selected">English</div></a>,
 <a href="/hi/read"><div class="lang-hi">हिन्दी</div></a>,
 <a href="/en/read" onclick="track_GA_Mixpanel({'hitType': 'event', 'category': 'LinkToAllNews', 'action': 'clicked', 'label': 'RedirectedToAllNews' });"> <li class="active-category selected">All News</li> </a>,
 <a href="/en/read/national" onclick="track_GA_Mixpanel({'hitType': 'event', 'category': 'LinkToIndiaNews', 'action': 'clicked', 'label': 'RedirectedToIndiaNews' });"> <li class="active-category">India</li> </a>,
 <a href="/en/read/business" onclick="track_GA_Mixpanel({'hitType': 'event', 'category': 'LinkToBusinessNews', 'action': 'clicked', 'label':  'RedirectedToBusinessNews' });"> <li class="active-category">Business</li> </a>,
 <a href="/en/read/sports" onclick="track_GA_Mixpanel({'hitType': 'event', 'category': 'LinkTosportsNews', 'acti

In [710]:
# #make url_list to cycle through the sites I want because it was too complicated the other way
# url_list = ['/business', '/sports', '/technology', '/entertainment']

In [711]:
# #cycle through list with timer to allow for load time printing response to ensure I have access
# url = 'https://inshorts.com/en/read'
# #create a dictionary to store soup variables for each endpoint
# soup_dict_ = {}
# response_list=[]
# for endpoint in url_list:
#     response = get(url)
#     print(f'{url}{endpoint}')
#     time.sleep(5)
#     print(response)
#     response_list.append(response)
#     print(f'access granted')

#     #iterate through the response_list and create a soup variable for each response
#     for endpoint, response in zip(url_list, response_list):
#         soup = BeautifulSoup(response.content, 'html.parser')
#         soup_dict_[endpoint.replace('/','')] = soup
#     print(f"soup_dict_{endpoint.replace('/','')}")
#     # Now you have a dictionary with keys as endpoints and values as the corresponding soup variables
#     print(f'soup made successfully')

In [712]:
#dictionary made encasing all soups
soup_dict_.keys()

dict_keys(['business', 'sports', 'technology', 'entertainment'])

In [713]:
#i still need to figure out what the class titles are called...

In [714]:
soup.find_all(class_='news-card-title')

[<div class="news-card-title news-right-box">
 <a class="clickable" href="/en/news/india-pakistan-footballers-break-into-fight-during-match-1687360048498" onclick="track_GA_Mixpanel({'hitType': 'event', 'category': 'TitleOfNews', 'action': 'clicked', 'label': 'India%2C%20Pakistan%20footballers%20break%20into%20fight%20during%20match)' });" style="color:#44444d!important">
 <span itemprop="headline">India, Pakistan footballers break into fight during match</span>
 </a>
 <div class="news-card-author-time news-card-author-time-in-title">
 <a href="/prev/en/news/india-pakistan-footballers-break-into-fight-during-match-1687360048498"><span class="short">short</span></a> by <span class="author">Anmol Sharma</span> / 
       <span class="time" content="2023-06-21T15:07:28.000Z" itemprop="datePublished">08:37 pm</span> on <span clas="date">21 Jun 2023,Wednesday</span>
 </div>
 </div>,
 <div class="news-card-title news-right-box">
 <a class="clickable" href="/en/news/was-never-interested-in-bei

In [715]:
# url = 'https://inshorts.com/en/read/technology'
# response = get(url)
# soup = BeautifulSoup(response.content, 'html.parser')
# soup.find_all(itemprop='headline')


# #title list for titles
# title_list = []

# #create variable
# titles = soup.find_all(itemprop='headline')

# #establishing range for loop
# num_titles = len(soup.find_all(itemprop='headline'))

# #using np.arange to iterate through 
# for i in np.arange(num_titles):
#     title_list.append(titles[i].text)
# title_list

In [716]:
#now I need to figure out how to get the content

In [717]:
#found the contents
soup.find_all(itemprop='articleBody')

[<div itemprop="articleBody">India and Pakistan footballers broke into a fight during later stages of the first half of their 2023 SAFF Football Championship match. The fight erupted after India manager Igor Štimac attempted to grab football from a Pakistan player. Match officials intervened to pacify the situation. Štimac was shown a red card, while Pakistan manager Shahzad Anwar received a yellow card.</div>,
 <div itemprop="articleBody">NCP leader Ajit Pawar has asked the party to relieve him of the responsibilities of Leader of Opposition (LoP) in the Maharashtra Assembly, PTI reported. "I was never interested in working as Leader of Opposition, but accepted the role on demand of the party MLAs," he said. "Assign me any post in the party organisation," Pawar added.</div>,
 <div itemprop="articleBody">A 20-year-old Dalit woman was allegedly raped and killed by three men, including two police constables, in Rajasthan's Bikaner, police said. Both the policemen have been suspended imme

In [718]:
#create variable
contents = soup.find_all(itemprop='articleBody')
contents

[<div itemprop="articleBody">India and Pakistan footballers broke into a fight during later stages of the first half of their 2023 SAFF Football Championship match. The fight erupted after India manager Igor Štimac attempted to grab football from a Pakistan player. Match officials intervened to pacify the situation. Štimac was shown a red card, while Pakistan manager Shahzad Anwar received a yellow card.</div>,
 <div itemprop="articleBody">NCP leader Ajit Pawar has asked the party to relieve him of the responsibilities of Leader of Opposition (LoP) in the Maharashtra Assembly, PTI reported. "I was never interested in working as Leader of Opposition, but accepted the role on demand of the party MLAs," he said. "Assign me any post in the party organisation," Pawar added.</div>,
 <div itemprop="articleBody">A 20-year-old Dalit woman was allegedly raped and killed by three men, including two police constables, in Rajasthan's Bikaner, police said. Both the policemen have been suspended imme

In [719]:
#length of contents - verifying same shape
num_content = len(soup.find_all(itemprop='articleBody'))
num_content

25

In [720]:
#content list for contents
content_list = []

#create variable
contents = soup.find_all(itemprop='articleBody')

#establishing range for loop
num_content = len(soup.find_all(itemprop='articleBody'))

#using np.arange to iterate through 
for i in np.arange(num_content):
    content_list.append(contents[i].text)
content_list

['India and Pakistan footballers broke into a fight during later stages of the first half of their 2023 SAFF Football Championship match. The fight erupted after India manager Igor Štimac attempted to grab football from a Pakistan player. Match officials intervened to pacify the situation. Štimac was shown a red card, while Pakistan manager Shahzad Anwar received a yellow card.',
 'NCP leader Ajit Pawar has asked the party to relieve him of the responsibilities of Leader of Opposition (LoP) in the Maharashtra Assembly, PTI reported. "I was never interested in working as Leader of Opposition, but accepted the role on demand of the party MLAs," he said. "Assign me any post in the party organisation," Pawar added.',
 "A 20-year-old Dalit woman was allegedly raped and killed by three men, including two police constables, in Rajasthan's Bikaner, police said. Both the policemen have been suspended immediately and the matter is being investigated. Police said the victim and the key accused we

In [721]:
# #make the category section now - hard code it
# category_list=[]

In [722]:
# #hmm, this might work? try with business url
# b_url = url+'/business'
# b_response = get(url)
# soup = BeautifulSoup(b_response.content, 'html.parser')
# soup.find_all('category')

#did not work - abort using dictionary method

In [723]:
def get_news_articles():
    '''
    this function gets the top 25 news articles titles, content, and category from inshorts.com
    category is only business, sports, technology, and entertainment
    '''
    file = 'news_articles.json'
    
    if os.path.exists(file):
        with open(file) as f:
            return json.load(f)
            print('file found and loaded')
    print(f'file not found. scraping and writing to json file')
    
    #create a dictionary and a list to store variables
    soup_dict_ = {} #stores soup calls for each of the endpoints in url_list
    response_list = [] #stores the responses to iterate
    title_list = []
    content_list = []
    category_list = []

    #make url_list to cycle through the sites I want because it was too complicated the other way
    url_list = ['/business', '/sports', '/technology', '/entertainment']

    #access the website - not blocked, accepts python requests
    url = 'https://inshorts.com/en/read'
    response = get(url)

    #cycle through list with timer to allow for load time printing response to ensure I have access
    for endpoint in url_list:
        response = get(url)
        print(f'{url}{endpoint}')
        time.sleep(5)
        print(response)
        response_list.append(response)
        print(f'access granted')

        #iterate through the response_list and create a soup variable for each response
        for endpoint, response in zip(url_list, response_list):
            soup = BeautifulSoup(response.content, 'html.parser')
            soup_dict_[endpoint.replace('/','')] = soup

            #create variables
            titles = soup.find_all(itemprop='headline')
            contents = soup.find_all(itemprop='articleBody')

            #establishing range for loop
            num_titles = len(soup.find_all(itemprop='headline'))
            num_content = len(soup.find_all(itemprop='articleBody'))
            num_category = len(soup.find_all(itemprop='articleBody'))

        #using np.arange to iterate through 
        for i in np.arange(num_titles):
            title_list.append(titles[i].text)
        for i in np.arange(num_content):
            content_list.append(contents[i].text)
        for i in np.arange(num_category):
            category_list.append(endpoint.replace('/',''))

        #combine lists to one dictionary following assigned format
        results = {'title':title_list,
                   'content':content_list,
                   'category':category_list}
        
    results_df = pd.DataFrame(results)
    
    with open(file, 'w') as f:
        json.dump(results, f)
        
    return results_df

In [724]:
results_df = get_news_articles()
results_df

{'title': ['Was never interested in being LoP, give me party post: Ajit Pawar',
  "2 transgenders killed with stones & knives in Hyderabad's Daibagh",
  " I declare less than actual area while declaring properties: K'taka Deputy CM",
  'Experience hassle-free home interiors: HomeLane',
  'Asit Modi would pinch my cheeks, say inappropriate things about my looks: Actress in FIR',
  'US overestimated value of weapons sent to Ukraine by $6.2 bn due to error: Pentagon',
  "Iceland Cricket troll England with 'Bazball 0-1 Basic Common Sense' tweet",
  "TIME releases list of the world's 100 most influential companies",
  "'Cis' or 'cisgender' are considered slurs on Twitter: Elon Musk",
  "Int'l Yoga day a call to stand up for humankind: NY Mayor at UN HQ",
  'Woman tortured to death by relatives in UP, loud music played to cover up screams',
  'Urge brave sisters to lead way to bring peace to Manipur: Sonia',
  'Yoga event led by PM Modi in US creates Guinness World Record',
  'Thought kids a

### Amanda's Way

In [739]:
#get access
response2 = get("https://inshorts.com/en/read/business")
soup2 = BeautifulSoup(response2.content, 'html.parser')
soup2.text[:400]

'\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nBusiness News: World and India Business News in English with Inshorts\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\ntoggle menuMenu\n\n\n\n\n\n\n\n\xa0\n\n\n\n\n\nEnglish\nहिन्दी\n\n\n\nCategories\n\n All News \n India \n Business \n Sports \n World \n Politics \n Technology \n Startup \n Entertainment \n Miscellaneous \n Hatke \n Science \n Automobile \n\n\n\n\n\n\n\n\n\n\n\n\n          For the best experienc'

In [740]:
#get titles
titles = soup2.find_all("span", itemprop="headline")
titles

[<span itemprop="headline">Sensex, Nifty end at fresh closing highs</span>,
 <span itemprop="headline">TIME releases list of the world's 100 most influential companies</span>,
 <span itemprop="headline">Which are the world's top 10 airlines according to passengers?</span>,
 <span itemprop="headline">Loves India, is a fan of PM: Paytm Founder on Musk after Modi meet</span>,
 <span itemprop="headline">Grab lays off over 1,000 employees</span>,
 <span itemprop="headline">India's tallest shopping mall to come up in Noida</span>,
 <span itemprop="headline">UK's net debt passes 100% of GDP for the first time since 1961</span>,
 <span itemprop="headline">Sylvester daCunha, the man behind iconic Amul girl, passes away</span>,
 <span itemprop="headline">McD employees, customer throw drinks at each other in Aus; company responds</span>,
 <span itemprop="headline">TCS wins $1-bn deal from UK's National Employment Savings Trust</span>,
 <span itemprop="headline">Yoga a global product made in India

In [741]:
#get summaries
summaries = soup2.find_all('div', itemprop="articleBody")
summaries[0]

<div itemprop="articleBody">Benchmark indices Sensex and Nifty ended at record closing highs on Wednesday. Sensex ended 195 points higher at 63,523 while the Nifty ended at 18,856.85, up 40 points. The gains were led by stocks like HDFC, Reliance Industries and TCS. During the intraday trade, Sensex rose to its fresh record high level of 63,588. </div>

In [742]:
#checkling lengths
len(titles), len(summaries)

(25, 25)

In [743]:
def scrape_one_page(topic):
#topic = "business"
    base_url = "https://inshorts.com/en/read/"

    response = get(base_url + topic)

    soup = BeautifulSoup(response.content, 'html.parser')

    titles = soup.find_all('span', itemprop="headline")

    summaries = soup.find_all('div', itemprop="articleBody")

    summary_list = []

    for i in range(len(titles)):
        temp_dict = {"category": topic,
                    "title": titles[i].text,
                    "content": summaries[i].text}

        summary_list.append(temp_dict)

    return summary_list

In [744]:
business_test = scrape_one_page("business")
business_test[0]

{'category': 'business',
 'title': 'Sensex, Nifty end at fresh closing highs',
 'content': 'Benchmark indices Sensex and Nifty ended at record closing highs on Wednesday. Sensex ended 195 points higher at 63,523 while the Nifty ended at 18,856.85, up 40 points. The gains were led by stocks like HDFC, Reliance Industries and TCS. During the intraday trade, Sensex rose to its fresh record high level of 63,588. '}

In [745]:
def get_news_articles(topic_list):
    """
    
    """
    file = "news_articles.json"
    if os.path.exists(file):
        with open(file) as f:
            return json.load(f)

    final_list = []

    for topic in topic_list:
        final_list.extend(scrape_one_page(topic))
    
    with open(file, "w") as f:
        json.dump(final_list, f)
        
    return final_list

In [748]:
topics = ['business', 'sports', 'technology', 'entertainment']

final_list = get_news_articles(topics)
final_list

{'title': ['Was never interested in being LoP, give me party post: Ajit Pawar',
  "2 transgenders killed with stones & knives in Hyderabad's Daibagh",
  " I declare less than actual area while declaring properties: K'taka Deputy CM",
  'Experience hassle-free home interiors: HomeLane',
  'Asit Modi would pinch my cheeks, say inappropriate things about my looks: Actress in FIR',
  'US overestimated value of weapons sent to Ukraine by $6.2 bn due to error: Pentagon',
  "Iceland Cricket troll England with 'Bazball 0-1 Basic Common Sense' tweet",
  "TIME releases list of the world's 100 most influential companies",
  "'Cis' or 'cisgender' are considered slurs on Twitter: Elon Musk",
  "Int'l Yoga day a call to stand up for humankind: NY Mayor at UN HQ",
  'Woman tortured to death by relatives in UP, loud music played to cover up screams',
  'Urge brave sisters to lead way to bring peace to Manipur: Sonia',
  'Yoga event led by PM Modi in US creates Guinness World Record',
  'Thought kids a

In [749]:
#checking
len(final_list)

3

In [751]:
# turn that bad boy into a df
final_df = pd.DataFrame(final_list)
pd.set_option("display.max_colwidth", None)
final_df

,title,content,category
0,"Was never interested in being LoP, give me party post: Ajit Pawar","NCP leader Ajit Pawar has asked the party to relieve him of the responsibilities of Leader of Opposition (LoP) in the Maharashtra Assembly, PTI reported. ""I was never interested in working as Leader of Opposition, but accepted the role on demand of the party MLAs,"" he said. ""Assign me any post in the party organisation,"" Pawar added.",business
1,2 transgenders killed with stones & knives in Hyderabad's Daibagh,"Two transgenders aged between 25 and 30 died after they were attacked with stones and knives in Hyderabad's Daibagh at around 1 am on Wednesday. Based on a preliminary investigation, police said the accused may have been intimately involved with them. In a separate incident, two homeless persons were killed with boulders when they were asleep on roadside in Hyderabad.",business
2,I declare less than actual area while declaring properties: K'taka Deputy CM,"Karnataka Deputy CM DK Shivakumar while talking about the payment of property taxes in Bengaluru on Wednesday said, ""When declaring my properties, I declare less than the actual (area)."" He added, ""Such things are there. People like us too are committing a lot of mistakes. All this should be streamlined."" He added that many people were not paying property taxes.",business
3,Experience hassle-free home interiors: HomeLane,"HomeLane invites you to discover the world of personalized interiors. With a '45-day delivery or we pay rent' guarantee and a 'flat 10-year warranty', HomeLane ensures a hassle-free experience. Powered by cutting-edge Spacecraft Pro software, you can get free personalized 3D designs and instant quotes with no hidden costs. HomeLane interiors start at just ₹3.5 lakh* with 0% EMI.",business
4,"Asit Modi would pinch my cheeks, say inappropriate things about my looks: Actress in FIR","The details of FIR filed against 'Taarak Mehta Ka Ooltah Chashmah' producer Asit Modi on basis of repeated complaints by an actress have been accessed by CNN-News18. In the FIR, the actress alleged he'd pinch her cheeks and say inappropriate things about her appearance. ""He'd also say...he would've hugged me if we were together while talking over...phone,"" the statement read.",business
...,...,...,...
95,Video shows Army personnel performing underwater yoga in Kerala,"On the occasion of ninth International Yoga Day, the Indian Army at Pangode Military Station in Kerala's Thiruvananthapuram organised an underwater yoga session. A video of the soldiers performing yoga underwater has surfaced online. It was performed by well-trained ""Amphibious Warriors"" of Pangode Military Station under the guidance of a yoga instructor and free diver.",entertainment
96,"PM Modi's visit is to boost ties, not about Russia or China: US","White House National Security Council Coordinator for Strategic Communications John Kirby said PM Narendra Modi's state visit to the country is not about China or Russia and is aimed at improving ties between the two countries. ""It's about focusing on where we are in this relationship,"" Kirby stated. PM Modi will address joint session of US Congress on June 22.\n",entertainment
97,German woman convicted of keeping Yazidi woman as 'slave' in Iraq,"A German woman has been convicted of keeping a Yazidi woman as a slave in Iraq and sentenced to over nine years in prison. A state court in Germany found that the defendant had abused the Yazidi woman for three years ""in her own interests"" as her husband, who was an Islamic State terrorist, regularly raped the latter.",entertainment
98,"Italians seeking surrogacy abroad may face up to €1 mn fine, jail","Italian lawmakers are debating a bill that could make it illegal for Italians to pursue parenthood abroad via surrogacy, even if the practice is legal in the concerned country. The proposed law would authorise prison terms of up to three years and fines of up to €1 million if a person is convicted. Surrogacy is